In [17]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
import pickle
import keras
from keras.datasets import cifar100
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, load_model
from keras.utils import to_categorical

In [2]:
# Training parameters
batch_size = 128
epochs = 200
num_classes = 100

data_augmentation = True
subtract_pixel_mean = True

In [3]:
# Model parameter (CIFAR-10)
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

version = 2
n = 2

In [4]:
depth = n * 9 + 2
model_type = 'ResNet%dv%d' % (depth, version)

In [5]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype(float)
x_test = x_test.astype(float)

if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [6]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [7]:
input_shape = x_train[0].shape

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000, 100)
y_test shape: (10000, 100)


In [8]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    
    return x

In [9]:
def resnet_v2(input_shape, depth, num_classes=100):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    elif epoch > 40:
        lr *= 5e-1
    print('Learning rate: ', lr)
    
    return lr

In [11]:
save_dir = os.path.join(os.getcwd(), 'saved_models/%s/baseline/' % model_type)
model_name = 'cifar100_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [12]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [13]:
model = resnet_v2(input_shape=input_shape, depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [14]:
if data_augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0
    )
else:
    datagen = ImageDataGenerator()

In [15]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                              validation_data=(x_test, y_test),
                              verbose=1, workers=4, callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
390/390 [==============================] - 21s 55ms/step - loss: 4.1884 - acc: 0.1249 - val_loss: 3.9548 - val_acc: 0.1468

Epoch 00001: val_acc improved from -inf to 0.14680, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet20v2/baseline/cifar100_ResNet20v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
390/390 [==============================] - 17s 42ms/step - loss: 3.5041 - acc: 0.2230 - val_loss: 3.9482 - val_acc: 0.1507

Epoch 00002: val_acc improved from 0.14680 to 0.15070, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet20v2/baseline/cifar100_ResNet20v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
390/390 [==============================] - 17s 43ms/step - loss: 3.1236 - acc: 0.2884 - val_loss: 3.3909 - val_acc: 0.2308

Epoch 00003: val_acc improved from 0.15070 to 0.23080, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet20v2/baseline/cifar100_ResNet20v2_model.003.h

390/390 [==============================] - 16s 42ms/step - loss: 1.3898 - acc: 0.6869 - val_loss: 1.9658 - val_acc: 0.5707

Epoch 00031: val_acc improved from 0.56230 to 0.57070, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet20v2/baseline/cifar100_ResNet20v2_model.031.h5
Epoch 32/200
Learning rate:  0.001
390/390 [==============================] - 16s 42ms/step - loss: 1.3704 - acc: 0.6935 - val_loss: 2.2160 - val_acc: 0.5322

Epoch 00032: val_acc did not improve from 0.57070
Epoch 33/200
Learning rate:  0.001
390/390 [==============================] - 16s 42ms/step - loss: 1.3591 - acc: 0.6991 - val_loss: 2.0338 - val_acc: 0.5628

Epoch 00033: val_acc did not improve from 0.57070
Epoch 34/200
Learning rate:  0.001
390/390 [==============================] - 16s 42ms/step - loss: 1.3416 - acc: 0.7010 - val_loss: 1.9009 - val_acc: 0.5803

Epoch 00034: val_acc improved from 0.57070 to 0.58030, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_model

390/390 [==============================] - 18s 46ms/step - loss: 0.7811 - acc: 0.8446 - val_loss: 1.8999 - val_acc: 0.6221

Epoch 00067: val_acc did not improve from 0.63940
Epoch 68/200
Learning rate:  0.0005
390/390 [==============================] - 18s 46ms/step - loss: 0.7736 - acc: 0.8479 - val_loss: 1.8826 - val_acc: 0.6233

Epoch 00068: val_acc did not improve from 0.63940
Epoch 69/200
Learning rate:  0.0005
390/390 [==============================] - 18s 46ms/step - loss: 0.7705 - acc: 0.8474 - val_loss: 1.8622 - val_acc: 0.6210

Epoch 00069: val_acc did not improve from 0.63940
Epoch 70/200
Learning rate:  0.0005
390/390 [==============================] - 18s 45ms/step - loss: 0.7594 - acc: 0.8520 - val_loss: 1.9432 - val_acc: 0.6236

Epoch 00070: val_acc did not improve from 0.63940
Epoch 71/200
Learning rate:  0.0005
390/390 [==============================] - 18s 45ms/step - loss: 0.7592 - acc: 0.8489 - val_loss: 2.0703 - val_acc: 0.6074

Epoch 00071: val_acc did not improve

390/390 [==============================] - 18s 46ms/step - loss: 0.4360 - acc: 0.9518 - val_loss: 1.8614 - val_acc: 0.6535

Epoch 00104: val_acc did not improve from 0.66370
Epoch 105/200
Learning rate:  0.0001
390/390 [==============================] - 18s 45ms/step - loss: 0.4326 - acc: 0.9519 - val_loss: 1.8840 - val_acc: 0.6465

Epoch 00105: val_acc did not improve from 0.66370
Epoch 106/200
Learning rate:  0.0001
390/390 [==============================] - 18s 45ms/step - loss: 0.4295 - acc: 0.9535 - val_loss: 1.8702 - val_acc: 0.6523

Epoch 00106: val_acc did not improve from 0.66370
Epoch 107/200
Learning rate:  0.0001
390/390 [==============================] - 18s 46ms/step - loss: 0.4292 - acc: 0.9529 - val_loss: 1.8874 - val_acc: 0.6546

Epoch 00107: val_acc did not improve from 0.66370
Epoch 108/200
Learning rate:  0.0001
390/390 [==============================] - 18s 46ms/step - loss: 0.4242 - acc: 0.9541 - val_loss: 1.8570 - val_acc: 0.6584

Epoch 00108: val_acc did not imp

390/390 [==============================] - 18s 46ms/step - loss: 0.3517 - acc: 0.9750 - val_loss: 1.8747 - val_acc: 0.6610

Epoch 00143: val_acc did not improve from 0.66370
Epoch 144/200
Learning rate:  1e-05
390/390 [==============================] - 18s 45ms/step - loss: 0.3516 - acc: 0.9753 - val_loss: 1.8771 - val_acc: 0.6604

Epoch 00144: val_acc did not improve from 0.66370
Epoch 145/200
Learning rate:  1e-05
390/390 [==============================] - 18s 45ms/step - loss: 0.3513 - acc: 0.9746 - val_loss: 1.8773 - val_acc: 0.6595

Epoch 00145: val_acc did not improve from 0.66370
Epoch 146/200
Learning rate:  1e-05
390/390 [==============================] - 18s 45ms/step - loss: 0.3530 - acc: 0.9739 - val_loss: 1.8797 - val_acc: 0.6612

Epoch 00146: val_acc did not improve from 0.66370
Epoch 147/200
Learning rate:  1e-05
390/390 [==============================] - 18s 45ms/step - loss: 0.3518 - acc: 0.9741 - val_loss: 1.8846 - val_acc: 0.6604

Epoch 00147: val_acc did not improve

390/390 [==============================] - 18s 46ms/step - loss: 0.3447 - acc: 0.9755 - val_loss: 1.8808 - val_acc: 0.6609

Epoch 00182: val_acc did not improve from 0.66370
Epoch 183/200
Learning rate:  5e-07
390/390 [==============================] - 18s 45ms/step - loss: 0.3420 - acc: 0.9771 - val_loss: 1.8804 - val_acc: 0.6603

Epoch 00183: val_acc did not improve from 0.66370
Epoch 184/200
Learning rate:  5e-07
390/390 [==============================] - 18s 45ms/step - loss: 0.3426 - acc: 0.9768 - val_loss: 1.8836 - val_acc: 0.6596

Epoch 00184: val_acc did not improve from 0.66370
Epoch 185/200
Learning rate:  5e-07
390/390 [==============================] - 18s 46ms/step - loss: 0.3433 - acc: 0.9754 - val_loss: 1.8797 - val_acc: 0.6598

Epoch 00185: val_acc did not improve from 0.66370
Epoch 186/200
Learning rate:  5e-07
390/390 [==============================] - 18s 46ms/step - loss: 0.3420 - acc: 0.9769 - val_loss: 1.8807 - val_acc: 0.6597

Epoch 00186: val_acc did not improve

In [18]:
history_filepath = os.path.join(save_dir, 'cifar100_%s_history_temp.pk' % (model_type))
pickle.dump(history, open(history_filepath, "wb"))

In [19]:
best_model_filepath = os.path.join(save_dir, 'cifar100_%s_model.%03d.h5' % (model_type, 91))
best_model = load_model(best_model_filepath)

scores = best_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 256us/step
Test loss: 1.7697390132904052
Test accuracy: 0.6637


In [20]:
def acc(top_k, y_true):
    correct = 0
    for i in range(y_true.shape[0]):
        if y_true[i] in top_k[i]:
            correct += 1
    return correct/float(y_true.shape[0])

y_pred = best_model.predict(x_test)
print("Accuracy: %f" % (np.sum(np.argmax(y_pred, axis=1) == np.argmax(y_test, axis=1)) / float(len(y_test))))

k = 5
y_pred_top_k = y_pred.argsort()[:,-k:]
print("Top %d accuracy: %f" % (k, acc(y_pred_top_k, np.argmax(y_test, axis=1))))

Accuracy: 0.663700
Top 5 accuracy: 0.893600


In [26]:
print("Extracting layer: %s" % (best_model.get_layer(index=-6).name))
feat_extractor_model = Model(inputs= best_model.input, outputs = best_model.get_layer(index=-6).output)

Extracting layer: add_6


In [27]:
x_train_feat = feat_extractor_model.predict(x_train)
print("train features:", x_train_feat.shape)
x_test_feat = feat_extractor_model.predict(x_test)
print("test features:", x_test_feat.shape)

train features: (50000, 8, 8, 256)
test features: (10000, 8, 8, 256)


In [28]:
save_dir_feat = os.path.join(os.getcwd(), 'saved_models/%s/extracted_feat/' % (model_type))
if not os.path.isdir(save_dir_feat):
    os.makedirs(save_dir_feat)
    
np.save(os.path.join(save_dir_feat, 'x_train_feat_cifar100_%s.npy' % (model_type)), x_train_feat)
np.save(os.path.join(save_dir_feat, 'x_test_feat_cifar100_%s.npy' % (model_type)), x_test_feat)